# A testing system for ´*.pt´-files

Based only on the `model.pt` file we can calculate stats from our standardized test set [Roboflow project](https://universe.roboflow.com/ntnu-3oxpl/testset-qs9tl).
This is just a hardcoded example set.
In order to use this test, just switch the `v12_larger.pt` file with your own model. Give the name in the box below and run the rest of the cells.

In [1]:
modelname = "v12_larger.pt"
pathToModel = f"data/models/{modelname}"

This is the solution of the test set:
It checks for the following things:
1. What color the house in focus  is.
2. How many houses each image has in total and their predictions


|Labels|white|red|orange|black|yellow|darkgrey|grey|brown|blue|green|pink|
|------|-----|---|------|-----|------|--------|----|-----|----|-----|----|
|Amount|21|9|8|6|6|3|3|2|1|1|1|

In [2]:
solution = []

# A hard-coded solution for the standard test set. First the main house, then all the present houses: 
solution.append(["Main house color", "other houses in picture"])

solution.append(["white", {'black':2, 'darkgrey':1, 'grey':1, 'white':1, 'yellow':1}])
solution.append(["orange", {'orange':1, 'red':1, 'white':1, 'yellow':1}])
solution.append(["red", {'orange':1, 'red':1, 'white':1}])
solution.append(["black", {'black':1, 'orange':1, 'yellow':1, 'white':1}])
solution.append(["empty", {}])
solution.append(["white", {'white':1}])
solution.append(["yellow", {'yellow':2}])
solution.append(["red", {'red':4, 'white':1}])
solution.append(["orange", {'orange':1}])
solution.append(["brown", {'brown':1, 'grey':1, 'red':1}])
solution.append(["white", {'white':3}])
solution.append(["darkgrey", {'darkgrey':1, 'white':1}])
solution.append(["grey", {'darkgrey':1, 'grey':1, 'red':1, 'white':1}])
solution.append(["black", {'orange':1, 'black':2, 'white':2, 'red':1}])
solution.append(["black", {'black':1}])
solution.append(["yellow", {'orange':1, 'green':1, 'white':2, 'yellow':1}])
solution.append(["white", {'white':4}])
solution.append(["brown", {'brown':1}])
solution.append(["blue", {'blue':1}])
solution.append(["white", {'orange':2, 'pink':1, 'white':1}])



In [3]:
# import modules
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image
import os

from tabulate import tabulate

Ultralytics YOLOv8.0.123  Python-3.10.1 torch-2.0.1+cpu CPU
Setup complete  (8 CPUs, 15.7 GB RAM, 160.4/475.7 GB disk)


In [4]:

model = YOLO(pathToModel)

### We need some functions in order to retrieve the results:

In [5]:
def gradeResults(result):
    predictions = []
    
    imgWidth = result[0].orig_shape[0]
    imgHeight = result[0].orig_shape[1]
    
    for box in result[0].boxes:
        predictedClass = model.names[int(box.cls)]
        confidence = round(float(box.conf), 3)
        
        x, y, w, h = box.xywh[0]
        x_offset = imgWidth/2 - (x)
        y_offset = imgHeight/2 - (y)
        offset =  x_offset**2 + y_offset**2
        predictions.append((predictedClass, confidence, offset))
    # Outputs list of guesses, confidence, and offset from center.
    return predictions

def chooseTheMiddleOne(predictions):
    if len(predictions) == 0:
        return "empty"
    # Sorts by offset from center
    predictions.sort(key=lambda x: x[2])
    
    minAcceptableOffset = 300
    if predictions[0][2] > minAcceptableOffset**2:
        return "bad"
    
    MiddleBoxClass = predictions[0][0]
    return MiddleBoxClass

### Predicting all the images in the folder

In [6]:
MyPredictions = [["Main house color", " other houses in picture"]]
for index in range(1, 21):
    img = "data/buildings/standardized_test/" + str(index) + ".png"
    result = model(img, conf=0.35)
    predictions = gradeResults(result)
    colors = dict()
    # Count the colors of the boxes
    for prediction in predictions:
        if prediction[0] in colors:
            colors[prediction[0]] += 1
        else:
            colors[prediction[0]] = 1    
    middleHousePrediction = chooseTheMiddleOne(predictions)
    
    MyPredictions.append([middleHousePrediction, colors])



image 1/1 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\standardized_test\1.png: 640x640 1 black, 1 blue, 2 darkgreys, 1 grey, 3 whites, 1202.4ms
Speed: 5.0ms preprocess, 1202.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\standardized_test\2.png: 640x640 2 oranges, 1 red, 1 white, 1072.4ms
Speed: 6.0ms preprocess, 1072.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\standardized_test\3.png: 640x640 2 browns, 1 orange, 1 white, 1163.1ms
Speed: 0.1ms preprocess, 1163.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\standardized_test\4.png: 640x640 1 black, 2 oranges, 1 white, 1150.1ms
Speed: 5.0ms preprocess, 1150.1ms inference, 0.0ms

In [7]:
# Printing a comparison table
print(tabulate(zip(MyPredictions, solution), headers=["The prediction", "The solution"]))

The prediction                                                            The solution
------------------------------------------------------------------------  --------------------------------------------------------------------------
['Main building', 'colors in picture']                                    ['Main house color', 'list of colors in picture']
['white', {'white': 3, 'black': 1, 'darkgrey': 2, 'blue': 1, 'grey': 1}]  ['white', {'black': 2, 'darkgrey': 1, 'grey': 1, 'white': 1, 'yellow': 1}]
['red', {'orange': 2, 'white': 1, 'red': 1}]                              ['orange', {'orange': 1, 'red': 1, 'white': 1, 'yellow': 1}]
['brown', {'white': 1, 'brown': 2, 'orange': 1}]                          ['red', {'orange': 1, 'red': 1, 'white': 1}]
['black', {'white': 1, 'orange': 2, 'black': 1}]                          ['black', {'black': 1, 'orange': 1, 'yellow': 1, 'white': 1}]
['black', {'black': 1}]                                                   ['empty', {}]
['bad', {'whi

In [11]:
class Stats:
    def __init__(self, pred_, sol_):
        self.pred = pred_
        self.sol = sol_
        
        self.numberOfBuildings = 61
        self.numberOfImages = 20
                
    def ComparePredAndSol(imgPred: dict, imgSol: dict):
        correct = 0
        missed = 0
        tooMany = 0
        
        totalBuildings = 0
        totalBuildingsPredicted = 0
        
        # Counting the total number of buildings predicted
        for key in imgPred:
            totalBuildingsPredicted += imgPred[key]
        
        # Counting the total number of buildings in the solution and comparing with prediction
        for key in imgSol:
            totalBuildings += imgSol[key]
            
            if key in imgPred:
                if imgPred[key] == imgSol[key]:
                    correct += imgSol[key]
                else:
                    correct += min(imgPred[key], imgSol[key])   
        
        missed = max(totalBuildings - totalBuildingsPredicted, 0)  
        tooMany = max(0, totalBuildingsPredicted - totalBuildings)  
        return correct, missed, tooMany   
                
    def calculateAccuracy(self):
        # Main building calculation
        self.CorrectMainBuildings = 0
        for index in range(1, 21):
            if self.pred[index][0] == self.sol[index][0]:
                self.CorrectMainBuildings += 1
                
        self.CorrectBuildings = 0
        self.MissedBuildings = 0
        self.TooManyBuildings = 0
        for index in range(1, self.numberOfImages + 1):
            # Adding stats one image at a time
            correct, missed, tooMany = Stats.ComparePredAndSol(self.pred[index][1], self.sol[index][1])
            self.CorrectBuildings += correct
            self.MissedBuildings += missed
            self.TooManyBuildings += tooMany
        
    def printStats(self):
        statTable = []
        
        mainBuildAccStat = str(100 * round(self.CorrectMainBuildings/self.numberOfImages, 3))
        accStat = str(100 * round(self.CorrectBuildings/self.numberOfBuildings, 3))
        missStat = str(100 * round(self.MissedBuildings/self.numberOfBuildings, 3))
        tooManyStat = str(100 * round(self.TooManyBuildings/self.numberOfImages, 3))
        
        statTable.append(["Main building accuracy", str(self.CorrectMainBuildings), f"{str(self.numberOfImages)} Images", mainBuildAccStat])
        statTable.append(["Accuracy", str(self.CorrectBuildings), f"{str(self.numberOfBuildings)} Buildings", accStat])
        statTable.append(["Missed", str(self.MissedBuildings), f"{str(self.numberOfBuildings)} Buildings", missStat])
        statTable.append(["Too many predictions", str(self.TooManyBuildings), f"{str(self.numberOfImages)} Images", tooManyStat])                 
        
        header = ["Statistic", "Result", "Out of", "Percentage"]
        
        print(tabulate(statTable, headers=header))
        
        # Alternative way of printing without the tabulate method
        # print(f"Main building accuracy: {str(self.CorrectMainBuildings)}/{str(self.numberOfImages)}, {str(round(self.CorrectMainBuildings/self.numberOfImages, 3))}%")
        # print(f"Accuracy: {str(self.CorrectBuildings)}/{str(self.numberOfBuildings)}, {str(round(self.CorrectBuildings/self.numberOfBuildings, 3))}%")
        # print(f"Missed: {str(self.MissedBuildings)}/{str(self.numberOfBuildings)}, {str(round(self.MissedBuildings/self.numberOfBuildings, 3))}%")
        # print(f"Too many: {str(self.TooManyBuildings)}/{str(self.numberOfImages)}, {str(round(self.TooManyBuildings/self.numberOfImages, 3))}%")
        

Stats returned from the test:
1. Accuracy of main house predictions
2. Accuracy of all house predictions
3. How many houses were missed
4. Images with too many guesses



In [12]:
stats = Stats(MyPredictions, solution)
stats.calculateAccuracy()
stats.printStats()

Statistic                 Result  Out of          Percentage
----------------------  --------  ------------  ------------
Main building accuracy         9  20 Images             45
Accuracy                      39  61 Buildings          63.9
Missed                         6  61 Buildings           9.8
Too many predictions           7  20 Images             35


To inspect the results, uncomment the following line
That is to inspect the boxes
The result should be shown inside a new folder named `outputs\standardized_test`	

In [10]:
#  confidence_threshold = 0.25
#  model("data/buildings/standardized_test/", conf=confidence_threshold, save=True, project="./outputs", name="standardized_test")


image 1/20 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\standardized_test\1.png: 640x640 1 black, 1 blue, 2 darkgreys, 1 grey, 3 whites, 1122.1ms
image 2/20 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\standardized_test\10.png: 640x640 1 brown, 1 grey, 1 white, 1129.4ms
image 3/20 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\standardized_test\11.png: 640x640 1 black, 1 grey, 2 whites, 1114.7ms
image 4/20 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\standardized_test\12.png: 640x640 1 blue, 1 darkgrey, 1 white, 1139.1ms
image 5/20 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\standardized_test\13.png: 640x640 1 black, 1 darkgrey, 1 red, 1169.0ms
image 6/20 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\standardized_test\14.png: 640x640 1 blue, 2 browns, 1 orange, 1 red, 1 white, 1214.9ms
image 7/20 c

[ultralytics.yolo.engine.results.Results object with attributes:
 
 boxes: ultralytics.yolo.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'black', 1: 'blue', 2: 'brown', 3: 'darkgrey', 4: 'green', 5: 'grey', 6: 'orange', 7: 'pink', 8: 'red', 9: 'white', 10: 'yellow'}
 orig_img: array([[[ 57,  74,  65],
         [ 65,  81,  72],
         [ 72,  87,  79],
         ...,
         [208, 211, 207],
         [209, 211, 207],
         [205, 206, 203]],
 
        [[ 56,  73,  64],
         [ 58,  74,  66],
         [ 63,  79,  70],
         ...,
         [211, 213, 209],
         [208, 210, 207],
         [204, 205, 202]],
 
        [[ 57,  74,  66],
         [ 54,  71,  62],
         [ 56,  73,  64],
         ...,
         [209, 211, 207],
         [203, 204, 201],
         [197, 198, 196]],
 
        ...,
 
        [[179, 176, 176],
         [178, 176, 176],
         [178, 175, 176],
         ...,
         [224, 223, 225],
         [224, 223, 225],
   